In [1]:
!pip install pandas
!pip install matplotlib
!pip install spicy
!pip install requests


Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'C:\Program Files\Python39\python.exe -m pip install --upgrade pip' command.


Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'C:\Program Files\Python39\python.exe -m pip install --upgrade pip' command.


Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'C:\Program Files\Python39\python.exe -m pip install --upgrade pip' command.


Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'C:\Program Files\Python39\python.exe -m pip install --upgrade pip' command.


In [2]:
!pip install fastapi
!pip install uvicorn 
!pip install nest_asyncio

Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'C:\Program Files\Python39\python.exe -m pip install --upgrade pip' command.


Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'C:\Program Files\Python39\python.exe -m pip install --upgrade pip' command.


Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'C:\Program Files\Python39\python.exe -m pip install --upgrade pip' command.


In [26]:
!pip install beautifulsoup4

Defaulting to user installation because normal site-packages is not writeable
     ------------------------------------ 128.2/128.2 KB 503.9 kB/s eta 0:00:00



You should consider upgrading via the 'C:\Program Files\Python39\python.exe -m pip install --upgrade pip' command.


In [2]:
import datetime as dt, numpy as np,pandas as pd, matplotlib.pyplot as plt
from scipy.stats import norm
import requests
from fastapi import FastAPI
import nest_asyncio
import uvicorn
from fastapi.middleware.cors import CORSMiddleware

In [3]:
from bs4 import BeautifulSoup

In [4]:
app = FastAPI()

origins = [
    "http://localhost:4200"
]
app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)



@app.get("/PortfeuilleVar")
def port_VaR(stock_return,exposure,CI,days):
    stocks_return=pd.DataFrame(stock_return)
    stocks_return.dropna(axis=1,inplace=True)
    stock_return=stock_return.T

    stock_return=pd.DataFrame.pct_change(stock_return)
    
    stock_return = stock_return.drop(labels=0, axis=0)
    
    stock_vol=stock_return.std()
    CL = norm.ppf(CI/100)

   
    #Portfolio variance covariance matrice
    stock_cov = np.array(np.cov(np.array(stock_return.T)))
    print(stock_cov)
    #Stock poids

    poids = np.array(exposure)/sum(exposure)
    poids_mat = np.mat(poids)
    
    #Portfolio variance
       
    por_var = (poids_mat*stock_cov) * poids_mat.T
    print(por_var)
    
    #Portfolio VaR

    por_VaR = np.sqrt(por_var)* CL *np.sqrt(days)
     
    return por_VaR[0,0]



In [5]:

@app.get("/PortfeuilleVar/{id}/{CI}/{days}")
def port_VaR(id,CI:int,days:int):

    api_url = "http://localhost:8083/JMLessous/getCapitals/"+str(id)
    response = requests.get(api_url)
    exposure=response.json()

    api_url = "http://localhost:8083/JMLessous/portfeuilleCloses/"+str(id)
    response = requests.get(api_url)
    stocks_return=pd.DataFrame(response.json())
    stocks_return=pd.DataFrame(stocks_return)
    stocks_return.dropna(axis=1,inplace=True)
    stocks_return=stocks_return.T

    stocks_return=pd.DataFrame.pct_change(stocks_return)
    
    stocks_return = stocks_return.drop(labels=0, axis=0)
    
    stock_vol=stocks_return.std()
    CL = norm.ppf(CI/100)

   
    #Portfolio variance covariance matrice
    stock_cov = np.array(np.cov(np.array(stocks_return.T)))
    print(stock_cov)
    #Stock poids

    poids = np.array(exposure)/sum(exposure)
    poids_mat = np.mat(poids)
    
    #Portfolio variance
       
    por_var = (poids_mat*stock_cov) * poids_mat.T
    print(por_var)
    
    #Portfolio VaR

    por_VaR = np.sqrt(por_var)* CL *np.sqrt(days)
     
    return por_VaR[0,0]



api_url = "http://localhost:8083/JMLessous/portfeuilleCloses/2"
response = requests.get(api_url)
#stocks_return= pd.DataFrame()
stocks_return=pd.DataFrame(response.json())
stocks_return.dropna(axis=1,inplace=True)
stocks_return.T




api_url = "http://localhost:8083/JMLessous/getCapitals/2"
response = requests.get(api_url)

exposure=response.json()

exposure

In [6]:
@app.get("/getAllActu")
def getAllActu():
    url = "https://www.ilboursa.com/"

    ilboursa = requests.get(url)
    doc = BeautifulSoup(ilboursa.text,"html.parser")
    tags = doc.find_all(["div"],class_="alct")
    date_tags = doc.find_all(["div"],class_="rubDateLf")
    dates = []
    liens = []
    titres = []

    for date in date_tags :
        dates.append(date.text.strip())

    for tag in tags :
        liens.append(tag.findChild("a")["href"])
        titres.append(tag.findChild("a").text)
        

    actualites = list(zip(dates,liens,titres))
    print(actualites)
    
    return actualites

In [7]:
@app.get("/getProdActu/{ticker}")
def getProdActu(ticker:str):
    print(ticker)
    url = "https://www.ilboursa.com/marches/cotation_"+ticker
    
    print(url)
    ilboursaProd = requests.get(url)
    doc = BeautifulSoup(ilboursaProd.text,"html.parser")
    div_tag = doc.find(["div"],class_="lh25")
    dates = []
    liens = []
    titres = []
    #print(div_tag.findChildren("a"))
    for tag in div_tag.findChildren("span") :
        dates.append(tag.text.strip())

    for tag in div_tag.findChildren("a") :
        liens.append(tag["href"])
        titres.append(tag.text)
            

    actualites = list(zip(dates,liens,titres))
    print(actualites)
    
    return actualites

In [8]:
nest_asyncio.apply()
uvicorn.run(app, port=8000)

INFO:     Started server process [9340]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:60632 - "GET /getAllActu/ HTTP/1.1" 307 Temporary Redirect
[('30 décembre - 16:43', 'marches/le-tmm-du-mois-de-decembre-recule-legerement-a-7-26_38563', 'Le TMM du mois de décembre recule légèrement à 7,26%'), ('30 décembre - 11:14', 'marches/la-bct-releve-son-taux-directeur-et-le-taux-de-remuneration-de-lepargne-de-75-points_38560', 'La BCT relève son taux directeur et le taux de rémunération de l’épargne de 75 points'), ('29 décembre - 19:55', 'marches/la-cloche-de-la-bourse-sonne-la-premiere-cotation-de-maghrebia-vie_38555', 'La cloche de la Bourse sonne la première cotation de Maghrebia Vie'), ('28 décembre - 08:11', 'marches/sonede-international-augmentation-de-capital-realisee-dans-la-limite-de-85_38553', 'SONEDE International : Augmentation de capital réalisée dans la limite de 85%')]
INFO:     127.0.0.1:60632 - "GET /getAllActu HTTP/1.1" 200 OK
INFO:     127.0.0.1:60749 - "OPTIONS /getProdActu/AB HTTP/1.1" 200 OK
AB
https://www.ilboursa.com/marches/cotation_

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [9340]


n_sims = 1000000
rfr = 0
time = 30
vol = 0.25
s0 = 1

d= (rfr - 0.5 * vol**2)*(time/252)
a= vol * np.sqrt(time/252)
r = np.random.normal(0,1,(n_sims,1))
GBM_returns = s0 * np.exp(d + a*r)

GBM_returns

pers = [0.01, 0.1, 1.0, 2.5, 5.0,10.0]
var = scs.scoreatpercentile(GBM_returns - 1,pers)
var

df = pd.DataFrame(var,pers,columns=['Var'])
df